# Step 1 - Install the required dependencies and make sure the python version is 3.10 and above

In [ ]:
%pip install zenoml

In [ ]:
%pip install datasets
%pip install transformers
%pip install tqdm
%pip install torch

In [ ]:
!python --version

# Step 2 - Load a dataset from Hugging Face

In [1]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("cardiffnlp/tweet_eval", "sentiment")
df = pd.DataFrame(ds['test']).head(500)
df.head(5)

C:\Users\djoc8\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,text,label
0,@user @user what do these '1/2 naked pics' hav...,1
1,OH: “I had a blue penis while I was this” [pla...,1
2,"@user @user That's coming, but I think the vic...",1
3,I think I may be finally in with the in crowd ...,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",0


In [2]:
def label_map(x):
    if x == 0:
        return 'negative'
    elif x == 1:
        return 'neutral'
    elif x == 2:
        return 'positive'
    return x
df['label'] = df['label'].map(label_map)

# Step 3 - Run model inference

Warning: This step is going to download two models of ~500MB each. 

**If you don't want to download the models, you can jump to step 4 and use the provided data in the repo instead.**

### Run inference with roberta

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
import tqdm

results = []
texts = df['text'].to_list()

## Depending on your machine, this should take around 1 minute
for text in tqdm.tqdm(texts):
    results.append(pipe(text))

In [5]:
df['roberta'] = [r[0]['label'] for r in results]
df['roberta_score'] = [r[0]['score'] for r in results]

### Run inference with gpt2

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="LYTinn/finetuning-sentiment-model-tweet-gpt2")

In [ ]:
import tqdm

results = []
texts = df['text'].to_list()

## Depending on your machine, this should take around 1 minute
for text in tqdm.tqdm(texts):
    results.append(pipe(text))

In [8]:
df['gpt2'] = [r[0]['label'] for r in results]
df['gpt2_score'] = [r[0]['score'] for r in results]

## map labels back
def label_map(x):
    if x == 'LABEL_0':
        return 'negative'
    elif x == 'LABEL_1':
        return 'neutral'
    elif x == 'LABEL_2':
        return 'positive'
    return x
df['gpt2'] = df['gpt2'].map(label_map)

# Step 4 - Pre-processing data and add additional columns

In [9]:
## If you skip the model inference, uncomment the code below and load the provided data

# df = pd.read_csv('tweets.csv')

In [4]:
df["input_length"] = df["text"].str.len()

# Step 5 - Start Zeno for interactive slicing

In this step, you need to create 5 slices in the Zeno interface and derive meaningful insights.

As a starting point, try to create the two slices we provide:

1. Tweets with hashtags
2. Tweets with strong positive words (e.g., love) -- you can determine the exact words

Creating slices in Zeno is straightforward: Just click on the '+' button for 'create a new slice', and you can define the slice using existing column attributes, with simple value macthing or even regular expression.

![image.png](images/image.png)

There are more fun features in Zeno, including interactive metadata & model comparison -- feel free to check the teaser video in [README](https://github.com/zeno-ml/zeno) of the Zeno repository.

In [ ]:
%pip install zeno_client

In [ ]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

df = pd.read_csv('tweets.csv')
df = df.reset_index()

# Initialize a client with the API key.
client = ZenoClient('zen_86R3sKvKLptA6EizL7oLs-tm-iEzwoHk5ZE1SqQU-Vo')

project = client.create_project(
    name="Tweet Sentiment Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["correct"]),
    ]
)

project.upload_dataset(df, id_column="index", data_column="text", label_column="label")

models = ['roberta', 'gpt2']
for model in models:
    df_system = df[['index', model]]
    
    # Measure accuracy for each instance, which is averaged by the ZenoMetric above
    df_system["correct"] = (df_system[model] == df["label"]).astype(int)
    
    project.upload_system(df_system, name=model, id_column="index", output_column=model)


Successfully created project.
Access your project at  https://hub.zenoml.com/project/001f935d-a452-4288-8001-985882cf6f33/Tweet%20Sentiment%20Analysis


C:\Users\djoc8\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\zeno_client\util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85'
 '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99'
 '100' '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111'
 '112' '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123'
 '124' '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135'
 '1

Successfully uploaded data


100%|██████████| 1/1 [00:00<00:00,  1.26it/s]
C:\Users\djoc8\AppData\Local\Temp\ipykernel_5136\4235081219.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_system["correct"] = (df_system[model] == df["label"]).astype(int)


Successfully uploaded system


100%|██████████| 1/1 [00:00<00:00,  1.32it/s]

Successfully uploaded system


After running the code above, you should be able to access Zeno in http://localhost:8231


After successfully creating the two slices, come up with three *additional* slices you want to check and **create** the slices in the Zeno interface.

There are two directions to identify useful slices:
- Top-down: Think about what kinds of things the model can struggle with, and come up with some slices.
- Bottom-up: Look at model (mis-)predictions, come up with hypotheses, and translate them into data slices.

3. [YOUR CHOICE]
4. [YOUR CHOICE]
5. [YOUR CHOICE]

In [17]:
%pip install emoji

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   ---------------------------------------- 590.6/590.6 kB 7.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## Write down descriptions of additional slices you created

custom_slice_descriptions = [
    "Tweets that contain emojis.",
    "Tweets that contain mentions (i.e., '@' symbol).", 
    "Tweets Equal to 140 Length",
]

In [18]:
import pandas as pd
import emoji

def contiene_emoji(texto):
    return any(char in emoji.EMOJI_DATA for char in texto)

df1 = pd.read_csv('tweets.csv')
df1 = df1.reset_index()

df1['Tiene_Emoji'] = df1['text'].apply(contiene_emoji)

In [19]:
df1.head(25)

,index,text,label,roberta,roberta_score,input_length,gpt2,gpt2_score,Tiene_Emoji
0,0,@user @user what do these '1/2 naked pics' hav...,neutral,negative,0.804726,96,positive,0.913451,False
1,1,OH: “I had a blue penis while I was this” [pla...,neutral,neutral,0.866949,72,neutral,0.753405,False
2,2,"@user @user That's coming, but I think the vic...",neutral,neutral,0.763724,87,positive,0.999962,False
3,3,I think I may be finally in with the in crowd ...,positive,positive,0.774047,83,positive,0.898784,False
4,4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,neutral,0.416397,133,positive,0.986431,False
5,5,Savchenko now Saakashvili took drug test live ...,neutral,neutral,0.496369,103,negative,0.999930,False
6,6,How many more days until opening day? 😩,neutral,neutral,0.466571,39,negative,0.999988,True
7,7,Twitter's #ThankYouObama Shows Heartfelt Grati...,positive,positive,0.947964,59,positive,0.999593,False
8,8,All CSG and Fracking all around Australia is t...,neutral,negative,0.515992,112,positive,0.999992,False
9,9,@user @user @user @user @user @user take away ...,negative,neutral,0.646297,103,positive,0.792283,False


# Step 6 - Write down three addition data slices you want to create but do not have the metadata for slicing

In the previous step, you might have already come up with some slices you wanted to create but found it hard to do with existing metadata. Write down three of such slices in this step.

Example: 
- I want to create a slice on tweets using slangs
- I want to create a slice on non-English tweets (if any)

In [20]:
## Write down three additional data slices here:

additional_slice_descriptions = [
    "I want to create a slice on tweets with URLs",
    "I want to create a slice on tweets using bad words",
    "I want to create a slice on political tweets",
]

# Step 7 - Generate more test cases with Large Language Models

Select one slice from the three you wrote down and generate **10 test cases** using LLMs, which can include average case, boundary case, or difficult case.

Your input can be in the following format:

> Examples:
> - OH: “I had a blue penis while I was this” [playing with Google Earth VR]
> - @user @user That’s coming, but I think the victims are going to be Medicaid recipients.
> - I think I may be finally in with the in crowd #mannequinchallenge  #grads2014 @user
> 
> Generate more tweets using slangs.

The first part of **Examples** conditions the LLM on the style, length, and content of examples. The second part of **Instructions** instructs what kind of examples you want LLM to generate.

Use our provided GPTs to start the task: [llm-based-test-case-generator](https://chatgpt.com/g/g-982cylVn2-llm-based-test-case-generator). If you do not have access to GPTs, use the plain ChatGPT or other LLM providers you have access to instead.

In [21]:
## Write down the slice you select

slice_description = "Tweets that contain mentions (i.e., '@' symbol)."

## Write down all generated test cases here
#generate **10 test cases** using LLMs, which can include average case, boundary case, or difficult case

generated_test_cases = [
    # Average cases
    "Hey @user, how are you doing today?",
    "Check out this cool article by @tech_writer!",
    "Had a great time at the event with @friend1 and @friend2.",
    
    # Boundary cases
    "@user",
    "Hello @user!",
    "This is a test tweet without any mentions.",
    
    # Difficult cases
    "Email me at user@example.com",
    "Follow us on Twitter @company_handle for updates!",
    "Is this a mention? @",
    "Multiple mentions in one tweet: @user1 @user2 @user3"
]